In [2]:
import pandas as pd

In [3]:
retail_df = pd.read_excel("Online Retail Data Set.xlsx")

In [4]:
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
retail_df.shape

(541909, 8)

In [6]:
retail_df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [7]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [8]:
retail_df['InvoiceDate'] = pd.to_datetime(retail_df['InvoiceDate'], errors='coerce')

In [9]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [10]:
retail_df['Country'] = retail_df['Country'].astype('str')
retail_df['Description'] = retail_df['Description'].astype('str')

In [11]:
retail_df[['Country', 'Description']].dtypes

Country        object
Description    object
dtype: object

## phase 2 - data storage

In [12]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [13]:
load_dotenv()

True

In [14]:

db_url = os.getenv("DATABASE_URL")

In [15]:
engine = create_engine(db_url)
print("Connected to PostgreSQL successfully!")

Connected to PostgreSQL successfully!


In [16]:
import os
print(os.listdir())

['.env', '.git', 'LICENSE', 'Online Retail Data Set.xlsx', 'README.md', 'retail.ipynb']


In [17]:
retail_df.to_sql(
    'online_retail_raw',
    con=engine,
    schema='retail_data',
    if_exists='replace',
    index=False
)
print("DataFrame written to PostgreSQL table 'retail_data.online_retail_raw' successfully!")

DataFrame written to PostgreSQL table 'retail_data.online_retail_raw' successfully!


In [18]:
raw_df = pd.read_sql("SELECT * FROM retail_data.online_retail_raw;", con=engine)


## phase 3 - data preparation

In [20]:
query = "SELECT * FROM online_retail_raw LIMIT 5;"
sample_data = pd.read_sql(query, con=engine)
print(sample_data)

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [ ]:
raw_df.isnull().sum()

In [24]:
raw_df.duplicated().sum()

np.int64(5231)

In [25]:
raw_df[raw_df['Quantity']<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [26]:
raw_df[raw_df['UnitPrice'] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
623,536414,22139,nan,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1975,536545,21134,nan,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1976,536546,22145,nan,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1977,536547,37509,nan,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1992,536549,85226A,nan,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535340,581211,22142,check,14,2011-12-07 18:36:00,0.0,NaN,United Kingdom
536982,581234,72817,nan,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom
538505,581406,46000M,POLYESTER FILLER PAD 45x45cm,240,2011-12-08 13:58:00,0.0,NaN,United Kingdom
538506,581406,46000S,POLYESTER FILLER PAD 40x40cm,300,2011-12-08 13:58:00,0.0,NaN,United Kingdom


In [27]:
raw_df = raw_df[raw_df['UnitPrice'] >= 0]

In [30]:
raw_df = raw_df[raw_df['Quantity'] > 0]

In [31]:
raw_df = raw_df[~raw_df['InvoiceNo'].astype(str).str.startswith('C')]

In [32]:
raw_df = raw_df.dropna(subset=['CustomerID'])

In [33]:
raw_df = raw_df.drop_duplicates()

In [29]:
raw_df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [34]:
raw_df['Revenue'] = raw_df['Quantity'] * raw_df['UnitPrice']

In [35]:
raw_df['Year'] = raw_df['InvoiceDate'].dt.year
raw_df['Month'] = raw_df['InvoiceDate'].dt.month
raw_df['Day'] = raw_df['InvoiceDate'].dt.day

In [36]:
raw_df.to_sql(
    'online_retail_clean',
    con=engine,
    schema='retail_data',
    if_exists='replace',
    index=False
)

732

## phase 4 - data analysis

In [37]:
clean_df = pd.read_sql("SELECT * FROM retail_data.online_retail_clean;", con=engine)

In [38]:
df_2011 = clean_df[clean_df['Year'] == 2011]
monthly_rev = df_2011.groupby('Month')['Revenue'].sum()
print(monthly_rev)

Month
1      568101.310
2      446084.920
3      594081.760
4      468374.331
5      677355.150
6      660046.050
7      598962.901
8      644051.040
9      950690.202
10    1035642.450
11    1156205.610
12     517190.440
Name: Revenue, dtype: float64


In [39]:
non_uk = clean_df[clean_df['Country'] != 'United Kingdom']
country_revenue = non_uk.groupby('Country')['Revenue'].sum().sort_values(ascending=False).head(10)
print(country_revenue)

Country
Netherlands    285446.34
EIRE           265262.46
Germany        228678.40
France         208934.31
Australia      138453.81
Spain           61558.56
Switzerland     56443.95
Belgium         41196.34
Sweden          38367.83
Japan           37416.37
Name: Revenue, dtype: float64


In [40]:
country_qty = non_uk.groupby('Country')['Quantity'].sum().sort_values(ascending=False)

In [41]:
top_customers = clean_df.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False).head(10)
print(top_customers)

CustomerID
14646.0    280206.02
18102.0    259657.30
17450.0    194390.79
16446.0    168472.50
14911.0    143711.17
12415.0    124914.53
14156.0    117210.08
17511.0     91062.38
16029.0     80850.84
12346.0     77183.60
Name: Revenue, dtype: float64


In [43]:
country_demand = (
    clean_df.groupby("Country")["Quantity"]
    .sum()
    .reset_index()
    .rename(columns={"Quantity": "Total_Quantity"})
)

In [44]:
country_demand

,Country,Total_Quantity
0,Australia,84199
1,Austria,4881
2,Bahrain,260
3,Belgium,23237
4,Brazil,356
5,Canada,2763
6,Channel Islands,9485
7,Cyprus,6340
8,Czech Republic,671
9,Denmark,8235


In [48]:
country_demand = country_demand[country_demand["Country"] != "United Kingdom"]

In [50]:
country_demand["Rank"] = country_demand["Total_Quantity"].rank(
    method="dense", ascending=False
).astype(int)

country_demand = country_demand.sort_values("Total_Quantity", ascending=False)

In [42]:
global_demand = non_uk.groupby('Country')['Quantity'].sum().sort_values(ascending=False)
print(global_demand)

Country
Netherlands             200937
EIRE                    140383
Germany                 119156
France                  111429
Australia                84199
Sweden                   36078
Switzerland              30083
Spain                    27944
Japan                    26016
Belgium                  23237
Norway                   19338
Portugal                 16095
Finland                  10704
Channel Islands           9485
Denmark                   8235
Italy                     8112
Cyprus                    6340
Singapore                 5241
Austria                   4881
Israel                    4043
Poland                    3684
Canada                    2763
Iceland                   2458
USA                       2458
Unspecified               1785
Greece                    1557
United Arab Emirates       982
Malta                      970
Czech Republic             671
Lithuania                  652
European Community         499
Lebanon                    386


In [51]:
threshold = country_demand["Total_Quantity"].quantile(0.75)

country_demand["Opportunity_Level"] = country_demand["Total_Quantity"].apply(
    lambda x: "High Opportunity" if x >= threshold else "Normal"
)
country_demand

,Country,Total_Quantity,Rank,Opportunity_Level
23,Netherlands,200937,1,High Opportunity
10,EIRE,140383,2,High Opportunity
14,Germany,119156,3,High Opportunity
13,France,111429,4,High Opportunity
0,Australia,84199,5,High Opportunity
31,Sweden,36078,6,High Opportunity
32,Switzerland,30083,7,High Opportunity
30,Spain,27944,8,High Opportunity
19,Japan,26016,9,High Opportunity
3,Belgium,23237,10,Normal
